## Imports

In [1]:
import sys
sys.path.append("..")

import pandas as pd
import sqlalchemy
from sqlalchemy import select, extract
from sqlalchemy.sql.expression import and_

import config
from cyclops.orm import Database

2022-02-16 15:51:12,196 INFO config          - Log file is /mnt/nfs/home/krishnanam/log.log
2022-02-16 15:51:12,265 INFO cyclops.orm     - Log file is /mnt/nfs/home/krishnanam/log.log


## Load config, setup ORM

In [2]:
cfg = config.read_config("../configs/default/*.yaml")
db = Database(cfg)

2022-02-16 15:51:13,472 INFO cyclops.orm     - Database setup, ready to run queries!


## Extract slices

In [3]:
HOSPITAL = "SMH"
YEAR = 2020
LIMIT_LABS = 100000
LIMIT_MEDICATIONS = 100000
LIMIT_VITALS = 100000

## Query to get admin + diagnosis

In [4]:
query = select(db.public.ip_administrative.x,
               db.public.diagnosis.x
              ).where(and_(db.public.ip_administrative.hospital_id == HOSPITAL,
                           extract('year', db.public.ip_administrative.admit_date_time) == YEAR))\
              .join(db.public.diagnosis.x, 
                    db.public.ip_administrative.genc_id == db.public.diagnosis.genc_id
                   )
              
data = db.run_query(query)
print(f"{len(data)} rows fetched!")

2022-02-16 15:51:15,242 INFO cyclops.orm     - Query returned successfully, took 1.7409915924072266 s


15840 rows fetched!


## Query to get admin + labs

In [5]:
query = select(db.public.ip_administrative.x,
               db.public.lab.x
              ).where(and_(db.public.ip_administrative.hospital_id == HOSPITAL,
                           extract('year', db.public.ip_administrative.admit_date_time) == YEAR))\
              .join(db.public.lab.x, 
                    db.public.ip_administrative.genc_id == db.public.lab.genc_id
                   ).where(db.public.lab.lab_test_name_mapped != '').limit(LIMIT_LABS) # Use limit here, since lab queries can have millions of rows!
data = db.run_query(query)
print(f"{len(data)} rows fetched!")

2022-02-16 15:51:42,909 INFO cyclops.orm     - Query returned successfully, took 27.65257477760315 s


100000 rows fetched!


## Query to get admin + pharmacy

In [6]:
query = select(db.public.ip_administrative.x,
               db.public.pharmacy.x
              ).where(and_(db.public.ip_administrative.hospital_id == HOSPITAL,
                           extract('year', db.public.ip_administrative.admit_date_time) == YEAR))\
              .join(db.public.pharmacy.x, 
                    db.public.ip_administrative.genc_id == db.public.pharmacy.genc_id
                   ).limit(LIMIT_MEDICATIONS) # Use limit here, since pharmacy queries can have millions of rows!
data = db.run_query(query)
print(f"{len(data)} rows fetched!")

2022-02-16 15:51:51,486 INFO cyclops.orm     - Query returned successfully, took 8.550371646881104 s


76421 rows fetched!


## Query to get admin + vitals

In [7]:
query = select(db.public.ip_administrative.x,
               db.public.vitals.x
              ).where(and_(db.public.ip_administrative.hospital_id == HOSPITAL,
                           extract('year', db.public.ip_administrative.admit_date_time) == YEAR))\
              .join(db.public.vitals.x, 
                    db.public.ip_administrative.genc_id == db.public.vitals.genc_id
                   ).limit(LIMIT_VITALS)
data = db.run_query(query)
print(f"{len(data)} rows fetched!")

2022-02-16 15:51:54,131 INFO cyclops.orm     - Query returned successfully, took 2.545888900756836 s


100000 rows fetched!
